In [15]:
import pandas as pd
import sys
import numpy as np
import pathlib as pl
import zipfile
sys.path.append('../dependencies/')
sys.path.append('../scripts/')
import pyemu
from datetime import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import calendar
from postprocessing import plot_single_real
from post_process_model_output_fn import postprocess

In [16]:
# curr_model = '01473000'
curr_model = '05431486'
# curr_model = '09112500'
curr_root = 'ies_hot'
citer = 3

In [17]:
tmp_res_path = pl.Path(f'../results/{curr_model}.{curr_root}/')

In [22]:
postproc_nhm = True

if postproc_nhm:
    for cm in ['01473000','05431486','09112500']:
        print(f'postprocessing {cm}')
        c_path = pl.Path(f'../NHM_solutions/nhm_byHWobs_files/{curr_model}')
        postprocess(c_path, '.')

postprocessing 01473000
postprocessing 05431486
postprocessing 09112500


In [23]:
fig_dir = pl.Path(f'../postprocessing/figures/single_reals/{curr_model}')
if not fig_dir.exists():
    fig_dir.mkdir(parents=True)

In [24]:
res = pd.read_csv(tmp_res_path / f'{curr_root}.2.base.rei', delim_whitespace=True, skiprows=3, index_col=0)
res.loc[res.Measured<-999, 'Measured'] = np.nan
res.head(5)

,Group,Measured,Modelled,Residual,Weight
Name,,,,,
g_min_actet_mean_mon:10:36895,g_min_actet_mean_mon,0.029127,0.057157,-0.028029,13784.30196
g_min_actet_mean_mon:10:36899,g_min_actet_mean_mon,0.027509,0.059386,-0.031877,13784.30196
g_min_actet_mean_mon:10:36901,g_min_actet_mean_mon,0.025045,0.023122,0.001924,13784.30196
g_min_actet_mean_mon:10:36902,g_min_actet_mean_mon,0.024773,0.051313,-0.026540,13784.30196
g_min_actet_mean_mon:10:36903,g_min_actet_mean_mon,0.024373,0.054701,-0.030328,13784.30196


In [25]:
res_nhm = pd.read_csv(f'../NHM_solutions/nhm_byHWobs_files/{curr_model}/modelobs.dat', delim_whitespace=True, index_col=0)
res_nhm.head(5)

,obsval
obsname,
l_max_actet_mon:2000_1:36895,0.012198
l_max_actet_mon:2000_1:36899,0.007670
l_max_actet_mon:2000_1:36901,0.003054
l_max_actet_mon:2000_1:36902,0.006320
l_max_actet_mon:2000_1:36903,0.004160


In [26]:
newindex=[]
for i in res.index:
    if 'streamflow_daily' not in i:
        newindex.append(i)
    else:
        tmp = i.split(':')
        tmp[0]='streamflow_daily'
        newindex.append((':').join(tmp))
res.index=newindex

In [27]:
res.head()

,Group,Measured,Modelled,Residual,Weight
g_min_actet_mean_mon:10:36895,g_min_actet_mean_mon,0.029127,0.057157,-0.028029,13784.30196
g_min_actet_mean_mon:10:36899,g_min_actet_mean_mon,0.027509,0.059386,-0.031877,13784.30196
g_min_actet_mean_mon:10:36901,g_min_actet_mean_mon,0.025045,0.023122,0.001924,13784.30196
g_min_actet_mean_mon:10:36902,g_min_actet_mean_mon,0.024773,0.051313,-0.026540,13784.30196
g_min_actet_mean_mon:10:36903,g_min_actet_mean_mon,0.024373,0.054701,-0.030328,13784.30196


In [28]:
res.loc[res.Group==res.Group.unique()[-4]]

,Group,Measured,Modelled,Residual,Weight
streamflow_daily:2000_10_10:054310157,streamflow_daily_low,0.91,0.607816,0.302184,0.216620
streamflow_daily:2000_10_10:05431016,streamflow_daily_low,4.70,1.863102,2.836898,0.460894
streamflow_daily:2000_10_10:05431017,streamflow_daily_low,6.10,2.716947,3.383053,0.355115
streamflow_daily:2000_10_10:05431022,streamflow_daily_low,2.10,13.946307,-11.846307,0.000000
streamflow_daily:2000_10_10:05431032,streamflow_daily_low,27.00,20.732280,6.267720,0.080230
...,...,...,...,...,...
streamflow_daily:2010_9_5:05431486,streamflow_daily_low,115.00,68.036729,46.963271,0.018837
streamflow_daily:2010_9_6:05431486,streamflow_daily_low,110.00,66.491901,43.508099,0.019693
streamflow_daily:2010_9_7:05431486,streamflow_daily_low,106.00,67.406496,38.593504,0.020436
streamflow_daily:2010_9_8:05431486,streamflow_daily_low,103.00,64.267929,38.732071,0.021031


In [29]:
res = res.merge(res_nhm.rename(columns={'obsval':'NHM_modelled'}), left_index=True, right_index=True)

In [30]:
res

,Group,Measured,Modelled,Residual,Weight,NHM_modelled
g_min_actet_mean_mon:10:36895,g_min_actet_mean_mon,0.029127,0.057157,-0.028029,13784.301960,0.057756
g_min_actet_mean_mon:10:36899,g_min_actet_mean_mon,0.027509,0.059386,-0.031877,13784.301960,0.057217
g_min_actet_mean_mon:10:36901,g_min_actet_mean_mon,0.025045,0.023122,0.001924,13784.301960,0.051946
g_min_actet_mean_mon:10:36902,g_min_actet_mean_mon,0.024773,0.051313,-0.026540,13784.301960,0.056040
g_min_actet_mean_mon:10:36903,g_min_actet_mean_mon,0.024373,0.054701,-0.030328,13784.301960,0.049554
...,...,...,...,...,...,...
streamflow_mon:2010_9:05431016,streamflow_mon,NaN,6.081874,-10005.081874,0.000000,16.757344
streamflow_mon:2010_9:05431017,streamflow_mon,NaN,7.816424,-10006.816424,0.000000,19.459771
streamflow_mon:2010_9:05431022,streamflow_mon,NaN,27.131440,-10026.131440,0.000000,25.100951
streamflow_mon:2010_9:05431032,streamflow_mon,73.620689,41.956414,31.664275,1.782749,39.303562


In [31]:
res.Group.unique()

array(['g_min_actet_mean_mon', 'g_min_actet_mon', 'g_min_recharge_ann',
       'g_min_runoff_mon', 'g_min_sca_daily', 'g_min_soil_moist_ann',
       'g_min_soil_moist_mon', 'l_max_actet_mean_mon', 'l_max_actet_mon',
       'l_max_recharge_ann', 'l_max_runoff_mon', 'l_max_sca_daily',
       'l_max_soil_moist_ann', 'l_max_soil_moist_mon',
       'streamflow_nodata', 'streamflow_daily_large_asc',
       'streamflow_daily_large_dsc', 'streamflow_daily_small_asc',
       'streamflow_daily_small_dsc', 'streamflow_daily_pulse_asc',
       'streamflow_daily_pulse_dsc', 'streamflow_daily_low',
       'streamflow_daily_exlow', 'streamflow_mean_mon', 'streamflow_mon'],
      dtype=object)

In [35]:
# cgroup = 'actet_mean_mon'
[plot_single_real(res, cgroup, citer, curr_root, fig_dir, curr_model) for cgroup in [
    # 'actet_mean_mon',
    # 'actet_mon',
    # 'recharge_ann',
    # 'soil_moist_mon',
    # 'soil_moist_ann',
    # 'runoff_mon',
    # 'streamflow_daily',
    # 'streamflow_mean_mon',
    'streamflow_mon'
]]

[None]

In [34]:
res

,Group,Measured,Modelled,Residual,Weight,NHM_modelled
g_min_actet_mean_mon:10:36895,g_min_actet_mean_mon,0.029127,0.057157,-0.028029,13784.301960,0.057756
g_min_actet_mean_mon:10:36899,g_min_actet_mean_mon,0.027509,0.059386,-0.031877,13784.301960,0.057217
g_min_actet_mean_mon:10:36901,g_min_actet_mean_mon,0.025045,0.023122,0.001924,13784.301960,0.051946
g_min_actet_mean_mon:10:36902,g_min_actet_mean_mon,0.024773,0.051313,-0.026540,13784.301960,0.056040
g_min_actet_mean_mon:10:36903,g_min_actet_mean_mon,0.024373,0.054701,-0.030328,13784.301960,0.049554
...,...,...,...,...,...,...
streamflow_mon:2010_9:05431016,streamflow_mon,NaN,6.081874,-10005.081874,0.000000,16.757344
streamflow_mon:2010_9:05431017,streamflow_mon,NaN,7.816424,-10006.816424,0.000000,19.459771
streamflow_mon:2010_9:05431022,streamflow_mon,NaN,27.131440,-10026.131440,0.000000,25.100951
streamflow_mon:2010_9:05431032,streamflow_mon,73.620689,41.956414,31.664275,1.782749,39.303562
